In [1]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/imports.ipynb")
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/gathering_affinities.ipynb")

/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/nbformat/current.py:19: UserWarning: IPython.nbformat.current is deprecated.

- use IPython.nbformat for read/write/validate public API
- use IPython.nbformat.vX directly to composing notebooks of a particular version

  """)
/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


Response to a reviewer comment about creating a control set of residues in frequently mutated non-cancer genes <br>
We will use TTN and olfactory receptor genes to statisfy this comment

### Non-cancer genes

In [3]:
# establish a list of cancer genes: CGC and haploinsufficency 
cgc = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/references/CGC.csv')
cgc_genes = list(cgc['Gene Symbol'])
haplo_genes = [x.strip() for x in open('/cellar/users/ramarty/Data/hla/git_data/data/tumor_suppressor_genes.txt').readlines()] \
               + [x.strip() for x in open('/cellar/users/ramarty/Data/hla/git_data/data/onco_genes.txt').readlines()]

In [8]:
# combine into one set
cancer_genes = list(set(cgc_genes + haplo_genes))

In [6]:
# import set of all mutations
df_missense = pd.read_csv('/cellar/users/ramarty/Data/hla/mutations/processed_mutation_files.full_tcga.all_valid_muts.csv',
                          index_col=0)

In [9]:
# exclude all mutations in cancer genes
df_missense_non_cancer = df_missense[~df_missense.Hugo_Symbol.isin(cancer_genes)]

In [11]:
df_missense_non_cancer.head()

,Tumor_Sample_Barcode,Hugo_Symbol,Variant_Classification,Protein_position,Amino_acids,Chromosome,Start_Position,Strand,Reference_Allele,Tumor_Seq_Allele2,Transcript_ID,Barcode,combined
0,TCGA-FX-A3NJ-01A-11D-A21Q-09,INTS3,Missense_Mutation,500/1042,P/R,chr1,153761659,+,C,G,ENST00000318967,TCGA-FX-A3NJ,INTS3_P500R
3,TCGA-FX-A3NJ-01A-11D-A21Q-09,PLD5,Missense_Mutation,536/536,V/L,chr1,242089859,+,C,G,ENST00000442594,TCGA-FX-A3NJ,PLD5_V536L
7,TCGA-FX-A3NJ-01A-11D-A21Q-09,GC,Missense_Mutation,406/474,L/R,chr4,71754456,+,A,C,ENST00000273951,TCGA-FX-A3NJ,GC_L406R
9,TCGA-FX-A3NJ-01A-11D-A21Q-09,PSMG4,Missense_Mutation,65/123,T/A,chr6,3263702,+,A,G,ENST00000438998,TCGA-FX-A3NJ,PSMG4_T65A
15,TCGA-FX-A3NJ-01A-11D-A21Q-09,ACADSB,Missense_Mutation,138/432,F/V,chr10,123040574,+,T,G,ENST00000358776,TCGA-FX-A3NJ,ACADSB_F138V


In [13]:
# get high frequency mutations that are left and run through pipeline
non_cancer_vc = df_missense_non_cancer[['Barcode', 'combined']].drop_duplicates().combined.value_counts()

In [18]:
len(non_cancer_vc[non_cancer_vc > 4])

1598

In [21]:
non_cancer_mutations = list(non_cancer_vc[:1500].index)

In [29]:
with open('/cellar/users/ramarty/Data/hla_ii/presentation/residues/non_cancer.test.txt', 'w') as f:
    for mutation in non_cancer_mutations:
        f.write('{0}\n'.format(mutation))

###known non cancer

In [2]:
df_missense = pd.read_csv('/cellar/users/ramarty/Data/hla/mutations/processed_mutation_files.full_tcga.all_valid_muts.csv',
                          index_col=0)

In [3]:
putative = open('/cellar/users/ramarty/Data/hla_ii/generated_data/mutations/olfactory.csv').readline().split('\r')

In [4]:
olfactory = [x for x in putative if 'OR' in x]

In [5]:
olfactory = olfactory + ['TTN']

In [20]:
df_missense_olfactory = df_missense[df_missense.Hugo_Symbol.isin(olfactory)]

In [21]:
known_non_cancer = df_missense_olfactory[['Barcode', 'combined']].drop_duplicates().combined.value_counts()

In [27]:
len(known_non_cancer[known_non_cancer > 1])

1806

In [30]:
known_non_cancer_mutations = list(known_non_cancer[:1500].index)

In [31]:
with open('/cellar/users/ramarty/Data/hla_ii/presentation/residues/known_non_cancer.test.txt', 'w') as f:
    for mutation in known_non_cancer_mutations[:1500]:
        f.write('{0}\n'.format(mutation))

### Make matrix

In [ ]:
## see data_gathering/mutations/matrices.ipynb

###Export to table

In [7]:
non_cancer = pd.DataFrame({'Gene': olfactory})

In [8]:
from pandas import ExcelWriter
writer = ExcelWriter('/cellar/users/ramarty/Data/hla_ii/generated_data/tables/Established_non_cancer.xlsx')
non_cancer.to_excel(writer,'Established non-cancer genes', index=False)
writer.save()